# Create a function that receives one URL and scrape all 100 other urls connected via href

In [2]:
from utils import create_s3_connection
S3 = create_s3_connection()

2023-04-25 10:54:47 [info     ] Success connecting S3          endpoint=localhost:9010
2023-04-25 10:54:47 [info     ] Bucket at S3 already exists    bucket_name=dev-nlp-chatbot


In [3]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:
import re
import string
import random
import requests

from bs4 import BeautifulSoup

from unidecode import unidecode

def sanitize(text):
    text = text.lower()    
    # Remove any non-word characters from text
    text = re.sub(r'\W+', ' ', text)
    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    #Normalize
    text = unidecode(text)
    list_text = text.split()
    text = ' '.join([word for word in list_text if word.isalpha()])
    return text

def is_link_to_file(link: str):
    extensions = [
        ".pdf", ".doc", ".docx", ".ppt", ".pptx", ".xls", ".xlsx", ".txt",
        ".html", ".htm", ".css", ".js", ".xml", ".json", ".csv", ".tsv",
        ".zip", ".tar", ".gz", ".rar", ".7z", ".bz2", ".apk", ".exe",
        ".bmp", ".jpg", ".jpeg", ".png", ".gif", ".tif", ".tiff", ".svg",
        ".mp3", ".wav", ".ogg", ".flac", ".aac", ".m4a", ".mp4", ".avi",
        ".mkv", ".mov", ".wmv", ".flv", ".webm", ".srt", ".sub", ".ass",
        ".pdf", ".epub", ".mobi", ".azw", ".azw3", ".djvu", ".fb2", ".ibook"
    ]
    return any(ext in link for ext in extensions)
    
def filter_link(link: str):
    if not link:
        return None
    
    if not (link.startswith('http://') or link.startswith('https://')):
        return None
    
    filter_list = ['facebook', 'twitter', 'youtube', 'instagram', 'linkedin', 'github']
    if (any(site in link for site in filter_list)):
        return None
    
    if is_link_to_file(link):
        return None
    
    return link


def get_external_links(url):
    url = filter_link(url)
    if not url:
        return '', set([])
    
    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the response with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <a> tags with an href attribute that starts with 'http' or 'https'
    external_links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        filtered_link = filter_link(link=href)
        if filtered_link:
            external_links.append(filtered_link)

    all_text = soup.get_text(strip=True, separator=' ')
    all_text = sanitize(all_text)
    # Return the list of external links
    return all_text, set(external_links)


def crawl(url: str):
    # First website to crawl
    all_text, links = get_external_links(url)
    destination_fpath = f"html_content/{url.replace('https://', '').replace('http://', '').strip('/')}.txt"
    S3.save_html(content=all_text, fpath=destination_fpath)
    
    unique_links = list(set(links))
    if len(unique_links) < 1:
        print('Did not find links to crawl')
        return 
    
    crawl_children(urls=unique_links, n_sites=10, depth=2)
    return
    
def crawl_children(urls: list[str], n_sites: int, depth: int):
    # Crawl N children websites until depth == 0
    if depth == 0:
        return 
    
    try:
        urls = random.sample(urls, n_sites) 
    except:
        pass

    for idx,url in enumerate(urls):
        print('\nIdx: ', idx)
        all_text, links = get_external_links(url)
        destination_fpath = f"html_content/{url.replace('https://', '').replace('http://', '').strip('/')}.txt"
        S3.save_html(content=all_text, fpath=destination_fpath)
        print('\n')
        unique_links = list(set(links))
        pprint(unique_links)
        print(f'Save content at: {destination_fpath}')
        crawl_children(urls=unique_links, n_sites=5, depth=depth-1)

In [61]:
#url = 'https://realpython.com/build-a-chatbot-python-chatterbot/'
#url = 'https://www.receiteria.com.br/receitas-de-comidas-faceis-de-fazer/'
url = 'https://pt.wikipedia.org/wiki/Python'
crawl(url)


Idx:  0


['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://gl.wikipedia.org/wiki/Python',
 'https://stats.wikimedia.org/#/commons.wikimedia.org',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://petscan.wmflabs.org/?language=commons&categories=Python_(programming_language)&project=wikimedia&ns%5B6%5D=1',
 'https://en.wikiquote.org/wiki/en:Python',
 'https://id.loc.gov/authorities/sh96008834',
 'https://it.wikipedia.org/wiki/Python',
 'https



['https://tg.wikipedia.org/wiki/Python',
 'https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A4%BE%E0%A4%87%E0%A4%A5%E0%A4%A8',
 'https://jbo.wikipedia.org/wiki/paiton',
 'https://zh.wikipedia.org/wiki/Python',
 'https://gl.wikipedia.org/wiki/Python',
 'https://lmo.wikipedia.org/wiki/Python',
 'https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://nl.wik



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A4%BE%E0%A4%87%E0%A4%A5%E0%A4%A8',
 'https://jbo.wikipedia.org/wiki/paiton',
 'https://zh.wikipedia.org/wiki/Python',
 'https://gl.wikipedia.org/wiki/Python',
 'https://lmo.wikipedia.org/wiki/Python',
 'https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://nl.wikipedia.org/wiki/Python_(programmeertaal)'



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'http://www.amk.ca/python/writing/gvr-interview',
 'https://gl.wikipedia.org/wiki/Python',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://it.wikipedia.org/wiki/Python',
 'https://nqo.wikipedia.org/wiki/%DF%94%DF%8A%DF%8C%DF%95%DF%90%DF%B2%DF%AC',
 'https://lt.wikipedia.org/wiki/Python',
 'https://zh-classical.wikipedia.org/wiki/%E7%9A%AE%E5%90%8C',
 'https://www.mediawiki.org/',
 'https://hr.wikipe



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://www.python.org/downloads/release/python-3812/',
 'https://gl.wikipedia.org/wiki/Python',
 'https://an.wikipedia.org/wiki/Python',
 'https://www.python.org/downloads/release/python-3120a6/',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://it.wikibooks.org/wiki/Python',
 'https://www.python.org/news/',
 'https://www.python.org/dev/peps/pep-0569/',
 'https://it.wikipedia.org/wiki/Python',
 'https://golden.com/wiki/Python_%28programming_language%29-MNA48',
 'https://nqo.wikipedia.org/wiki/%DF

...



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'http://www.amk.ca/python/writing/gvr-interview',
 'https://gl.wikipedia.org/wiki/Python',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://zh-min-nan.wikipedia.org/w/index.php?title=Pang-b%C3%B4%CD%98:Programming_languages&action=edit',
 'https://it.wikipedia.org/wiki/Python',
 'https://nqo.wikipedia.org/wiki/%DF%94%DF%8A%DF%8C%DF%95%DF%90%DF%B2%DF%AC',
 'https://stats.wikimedia.org/#/zh-min-nan.wik



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://gl.wikipedia.org/wiki/Python',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://it.wikipedia.org/wiki/Python',
 'https://nqo.wikipedia.org/wiki/%DF%94%DF%8A%DF%8C%DF%95%DF%90%DF%B2%DF%AC',
 'https://lt.wikipedia.org/wiki/Python',
 'https://zh-classical.wikipedia.org/wiki/%E7%9A%AE%E5%90%8C',
 'https://www.mediawiki.org/',
 'https://www.python.org',
 'https://hr.wikipedia.org/wiki/Python_(pro



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://gl.wikipedia.org/wiki/Python',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://www.tiobe.com/tiobe-index/',
 'http://data.bnf.fr/ark:/12148/cb13560465c',
 'https://id.loc.gov/authorities/sh96008834',
 'https://it.wikipedia.org/wiki/Python',
 'https://nqo.wikipedia.org/wiki/%DF%94%DF%8A%DF%8C%DF%95%DF%90%DF%B2%DF%AC',
 'https://lt.wikipedia.org/wiki/Python',
 'https://zh-classical.wikipedia.



['https://www.python.org/',
 'http://www.cs.arizona.edu/icon/',
 'http://sf.net/cvs/?group_id=5470',
 'http://python.ca/nas/python/generator.diff',
 'http://www.stackless.com/']
Save content at: html_content/www.python.org/dev/peps/pep-0255.txt

Idx:  3


['https://tg.wikipedia.org/wiki/Python',
 'https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A4%BE%E0%A4%87%E0%A4%A5%E0%A4%A8',
 'https://jbo.wikipedia.org/wiki/paiton',
 'https://zh.wikipedia.org/wiki/Python',
 'https://gl.wikipedia.org/wiki/Python',
 'https://lmo.wikipedia.org/wiki/Python',
 'https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wi



['https://la.wikipedia.org/wiki/Python_(lingua_programmandi)',
 'https://gl.wikipedia.org/wiki/Python',
 'https://foundation.wikimedia.org/wiki/Cookie_statement',
 'https://an.wikipedia.org/wiki/Python',
 'https://te.wikipedia.org/wiki/%E0%B0%AA%E0%B1%88%E0%B0%A5%E0%B0%BE%E0%B0%A8%E0%B1%8D_(%E0%B0%95%E0%B0%82%E0%B0%AA%E0%B1%8D%E0%B0%AF%E0%B1%82%E0%B0%9F%E0%B0%B0%E0%B1%8D_%E0%B0%AD%E0%B0%BE%E0%B0%B7)',
 'https://da.wikipedia.org/wiki/Python_(programmeringssprog)',
 'https://mk.wikipedia.org/wiki/%D0%9F%D0%B0%D1%98%D1%82%D0%BE%D0%BD_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D1%81%D0%BA%D0%B8_%D1%98%D0%B0%D0%B7%D0%B8%D0%BA)',
 'https://uk.wikipedia.org/wiki/Python',
 'https://www.python.org/downloads/',
 'https://it.wikipedia.org/wiki/Python',
 'https://nqo.wikipedia.org/wiki/%DF%94%DF%8A%DF%8C%DF%95%DF%90%DF%B2%DF%AC',
 'https://lt.wikipedia.org/wiki/Python',
 'https://zh-classical.wikipedia.org/wiki/%E7%9A%AE%E5%90%8C',
 'https://www.mediawiki.org/',
 'https://hr.wikipedia.org/wiki/

In [29]:
from pprint import pprint

pprint(list(dic_texts.keys()))

['br.pinterest.com/receiteria',
 'www.grupocontteudo.com.br',
 'www.receiteria.com.br',
 'www.receiteria.com.br/arroz-e-risotos',
 'www.receiteria.com.br/bebidas',
 'www.receiteria.com.br/bolos',
 'www.receiteria.com.br/carnes',
 'www.receiteria.com.br/como-fazer-leite-evaporado',
 'www.receiteria.com.br/como-fazer-pipoca-de-cinema',
 'www.receiteria.com.br/doces-e-sobremesas',
 'www.receiteria.com.br/entradas-e-petiscos',
 'www.receiteria.com.br/entrar',
 'www.receiteria.com.br/imprimir?id=142617',
 'www.receiteria.com.br/lanches-e-salgados',
 'www.receiteria.com.br/molhos-e-pates',
 'www.receiteria.com.br/paes',
 'www.receiteria.com.br/privacidade',
 'www.receiteria.com.br/receita/arroz-com-calabresa-na-pressao-facil',
 'www.receiteria.com.br/receita/batata-rustica',
 'www.receiteria.com.br/receita/biscoito-amanteigado-sem-fermento',
 'www.receiteria.com.br/receita/bisnaguinha-sem-gluten',
 'www.receiteria.com.br/receita/bolinho-de-arroz-recheado-com-queijo',
 'www.receiteria.com.br/

# Load content from S3

In [5]:
dic_texts = {}

for s3_path in S3.list_files_recursive(folder='html_content/'):
    key = s3_path.split('.txt')[0]
    value = S3.read_html(fpath='html_content/'+s3_path)
    dic_texts[key] = value
    
    
for k,v in dic_texts.items():
    print(f'Path: {k}\nContent: {v[:10]}\n')

Path: als.wikipedia.org/wiki/Python_(Programmiersprache)
Content: python pro

Path: ast.wikipedia.org/wiki/Portada
Content: wikipedia 

Path: az.wikipedia.org/wiki/Python_(proqramla%C5%9Fd%C4%B1rma_dili)
Content: python pro

Path: br.pinterest.com/receiteria
Content: receiteria

Path: ca.wikipedia.org/wiki/Python
Content: python viq

Path: ceb.wikipedia.org/wiki/Python_(programming_language)
Content: python pro

Path: code.google.com/p/unladen-swallow/wiki/ProjectPlan
Content: internal s

Path: commons.wikimedia.org/wiki/Special:Search/Category:Python_(programming_language)
Content: category p

Path: disqus.com/?ref_noscript
Content: disqus the

Path: el.wikipedia.org/wiki/Python
Content: python bik

Path: en.wikipedia.org/wiki/Non-profit_organization
Content: nonprofit 

Path: et.wikipedia.org/wiki/Python_(programmeerimiskeel)
Content: python pro

Path: eu.wikipedia.org/wiki/Python_(informatika)
Content: python inf

Path: fa.wikipedia.org/wiki/%D9%BE%D8%A7%DB%8C%D8%AA%D9%88%D9%86_(%D8

In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Build reverse-index

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def build_reverse_index(dic_texts: dict):
    content_from_urls = list(dic_texts.values())
    website_urls      = list(dic_texts.keys())
    
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(content_from_urls)

    res = {}

    for word in vectorizer.get_feature_names_out():
        j = vectorizer.vocabulary_[word]
        doc_df = {}
        
        for i in range(len(website_urls)):
            if tfidf[i,j] > 0:
                doc_df[website_urls[i]] = tfidf[i,j]
                
        res[word] = doc_df
    
    return res

reverse_index = build_reverse_index(dic_texts)

In [15]:
def search_words(list_words: list[str], rev_index, n: int):
    assert type(list_words)==list
    result = {}
    not_found = []
    
    for word in list_words:
        word = word.lower()
        if word in rev_index.keys():
            for documento in rev_index[word].keys():
                if documento not in result.keys():
                    result[documento]  = rev_index[word][documento]
                else:
                    result[documento] += rev_index[word][documento]
        else:
            not_found.append(word)
    
    if result:
        sorted_res = dict(sorted(result.items(), key=lambda x: x[1], reverse=True)[:n])
        return sorted_res, not_found
    
    return {}, not_found

search_words(list_words=['carne'], rev_index=reverse_index, n=5)

({'www.receiteria.com.br/carnes': 0.14227529655628074,
  'www.receiteria.com.br/receita/penne-com-frango': 0.08051617862138481,
  'www.receiteria.com.br': 0.08002302073683487,
  'www.receiteria.com.br/receita/maionese-de-ovo-cozido': 0.0732510728163526,
  'www.receiteria.com.br/receita/molho-de-shimeji': 0.07110451840364586},
 [])